In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from bioinf_common.plotting import annotated_barplot

In [ ]:
sns.set_context('talk')

# Parameters

In [ ]:
fname = snakemake.input.fname

outdir = Path(snakemake.output.outdir)

# Read data

In [ ]:
df_all = pd.read_csv(fname, dtype={'associated_genes': str})
df_all.head()

# Select data subset

In [ ]:
row = df_all.iloc[0]
df = df_all[(df_all['tad_source'] == row.tad_source) & (df_all['window_size'] == row.window_size)]

print(row.tad_source, row.window_size)
df.shape

In [ ]:
df_uniqsnp = df.drop_duplicates('snpId')

# Overview tables

In [ ]:
df_uniqsnp.head(1)

In [ ]:
pd.DataFrame({
    '#snps': [df_uniqsnp.shape[0]],
    '#border_snps': ((df_uniqsnp['TAD_20in'] == 'boundary')).sum(),
    '%border_snps': ((df_uniqsnp['TAD_20in'] == 'boundary')).sum() / df_uniqsnp.shape[0],
})

In [ ]:
pd.DataFrame({
    '#cancer_snps': [df_uniqsnp['is_cancer'].sum()],
    '#border_cancer_snps': (df_uniqsnp['is_cancer'] & (df_uniqsnp['TAD_20in'] == 'boundary')).sum(),
    '%border_cancer_snps': (df_uniqsnp['is_cancer'] & (df_uniqsnp['TAD_20in'] == 'boundary')).sum() / df_uniqsnp['is_cancer'].sum(),
})

In [ ]:
pd.DataFrame({
    '#cancer_snps': [df_uniqsnp['is_cancer'].sum()],
    '#intergenic_cancer_snps': (df_uniqsnp['is_cancer'] & (df_uniqsnp['variant_group_hg38'] == 'intergenic')).sum(),
    '%intergenic_cancer_snps': (df_uniqsnp['is_cancer'] & (df_uniqsnp['variant_group_hg38'] == 'intergenic')).sum() / df_uniqsnp['is_cancer'].sum(),
})

In [ ]:
pd.DataFrame({
    '#snps': [df_uniqsnp.shape[0]],
    '#intergenic_snps': (df_uniqsnp['variant_group_hg38'] == 'intergenic').sum(),
    '%intergenic_snps': (df_uniqsnp['variant_group_hg38'] == 'intergenic').sum() / df_uniqsnp.shape[0],
})

In [ ]:
pd.DataFrame({
    '#intergenic_snps': [(df_uniqsnp['variant_group_hg38'] == 'intergenic').sum()],
    '#border_intergenic_snps': ((df_uniqsnp['variant_group_hg38'] == 'intergenic') & (df_uniqsnp['TAD_20in'] == 'boundary')).sum(),
    '%border_intergenic_snps': ((df_uniqsnp['variant_group_hg38'] == 'intergenic') & (df_uniqsnp['TAD_20in'] == 'boundary')).sum() / (df_uniqsnp['variant_group_hg38'] == 'intergenic').sum(),
})

In [ ]:
pd.DataFrame({
    '#intergenic_cancer_snps': [(df_uniqsnp['is_cancer'] & (df_uniqsnp['variant_group_hg38'] == 'intergenic')).sum()],
    '#border_intergenic_cancer_snps': (df_uniqsnp['is_cancer'] & (df_uniqsnp['variant_group_hg38'] == 'intergenic') & (df_uniqsnp['TAD_20in'] == 'boundary')).sum(),
    '%border_intergenic_cancer_snps': (df_uniqsnp['is_cancer'] & (df_uniqsnp['variant_group_hg38'] == 'intergenic') & (df_uniqsnp['TAD_20in'] == 'boundary')).sum() / (df_uniqsnp['variant_group_hg38'] == 'intergenic').sum(),
})

# Plot database statistics

## Number of entries per disease

In [ ]:
disease_counts = (df['diseaseId']
                  .value_counts()
                  .rename('count')
                  .reset_index()
                  .rename(columns={'index': 'diseaseId'})
                  .sort_values('count')
                  .merge(df[['diseaseId', 'is_cancer']], how='left', on='diseaseId'))

disease_counts.head()

In [ ]:
sns.boxplot(x='is_cancer', y='count', data=disease_counts)

plt.title('#rows associated with single diseases')
plt.yscale('log')

plt.tight_layout()
plt.savefig(outdir / 'disease_count_distribution.pdf')

## Odds ratio distribution

In [ ]:
df['odds_ratio'].describe()

In [ ]:
odds_ratio = df['odds_ratio'].dropna()
sns.boxplot(odds_ratio[odds_ratio < odds_ratio.quantile(.75)], orient='v')

plt.title('Odds ratios (< 75% quantile) for all diseases')

plt.tight_layout()
plt.savefig(outdir / 'oddsratio_distribution.pdf')

## VEP statistics

### Raw variant types

In [ ]:
variant_type_col = df.filter(like='variant_type').columns[0]

In [ ]:
variant_type_counts = (df[['snpId', variant_type_col]]
                       .drop_duplicates()[variant_type_col]
                       .value_counts()
                       .rename('count')
                       .reset_index()
                       .rename(columns={'index': 'variant_type'}))

In [ ]:
plt.figure(figsize=(16, 8))
sns.barplot(
    x='count', y='variant_type',
    data=variant_type_counts, orient='h', color=sns.color_palette()[0])

plt.title('#variant_type in database')
plt.xscale('log')

plt.tight_layout()
plt.savefig(outdir / 'variant_type_counts.pdf')

### Variant groups

In [ ]:
variant_group_col = df.filter(like='variant_group').columns[0]

In [ ]:
variant_group_counts = (df[['snpId', variant_group_col]]
                       .drop_duplicates('snpId')[variant_group_col]
                       .value_counts()
                       .rename('count')
                       .reset_index()
                       .rename(columns={'index': 'variant_group'}))

In [ ]:
plt.figure(figsize=(8, 6))
annotated_barplot(
    x='variant_group', y='count', data=variant_group_counts,
    anno_kws=dict(label_offset=8, label_size=12))

plt.tight_layout()
plt.savefig(outdir / 'variant_group_counts.pdf')

## Gene counts

In [ ]:
df_tmp = pd.DataFrame({
    'diseaseId': df['diseaseId'],
    'associated_genes': df['associated_genes'].str.split(','),
    'gene_count': df['associated_genes'].str.split(',').apply(lambda x: len(x) if isinstance(x, list) else 0)
})
df_tmp.head()

In [ ]:
sns.boxplot(y=df_tmp.groupby('diseaseId')['gene_count'].sum())

plt.xlabel('All diseases')
plt.ylabel('#associated genes')

unique_genes = set(g for gs in df_tmp['associated_genes'] if isinstance(gs, list) for g in gs)
plt.title(f'{len(unique_genes)} unique genes in total')

plt.yscale('log')

plt.tight_layout()
plt.savefig(outdir / 'gene_counts.pdf')

## Filter statistics

In [ ]:
df_filter_stats = df.filter(like='filter_').sum(axis=0).to_frame('count').reset_index()
df_filter_stats.head()

In [ ]:
plt.figure(figsize=(16, 8))
sns.barplot(
    x='count', y='index', data=df_filter_stats,
    orient='h', color=sns.color_palette()[0])

plt.xlabel('Entry count')
plt.ylabel('Filter type')

plt.tight_layout()
plt.savefig(outdir / 'filter_counts.pdf')